# Graph Data structures

Topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC


Early start and early finish topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC

In [1]:
"""
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC

MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC
"""

SyntaxError: invalid syntax (175008303.py, line 1)

In [ ]:
"""
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)    
WITH activity, predecessor, COUNT(predecessor) AS incomingDependencies     

OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY) 
WITH activity, predecessor, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth 
ORDER BY depth ASC, incomingDependencies ASC   // activity with longest chain comes first
WITH activity, COLLECT(predecessor) AS predecessors                 // group all predecessor nodes into a list
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors |                    //  Iterate over each predecessor for forward pass take max of predecessors early finish 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish  // get max early finish of predecessors
            ELSE maxFinish 
        END) AS earlyStart                                          // the maxfinish in the predecessors will be the early start of current node  or 0 if no predecessors acc maxFinish = 0                     
SET activity += {earlyStart: COALESCE(earlyStart, 0),
    earlyFinish: COALESCE(earlyStart, 0) + activity.duration} 
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC;"""

In [ ]:
"""
// initialize earlystart and early finish of the last node
MATCH (activity:ACTIVITY)
WHERE NOT (activity)-[:DEPENDS_ON]->()  // Find node with no outgoing relationship
SET activity.latestFinish = activity.earlyFinish,
    activity.latestStart = activity.latestFinish - activity.duration;

// Propagate latestStart and latestFinish values backward
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY) // Match successors
WITH activity, 
     COLLECT(successor) AS successors                         // Group successors for each node
ORDER BY activity.depth DESC                                  // Process nodes in reverse order of depth
WITH activity,
     REDUCE(minStart = activity.latestFinish, succ IN successors | 
         CASE 
             WHEN succ.latestStart IS NOT NULL AND succ.latestStart < minStart THEN succ.latestStart
             ELSE minStart
         END) AS latestFinish                                 // Calculate latestFinish as min(latestStart) of successors
SET activity.latestFinish = COALESCE(latestFinish, activity.earlyFinish),
    activity.latestStart = activity.latestFinish - activity.duration;

// Step 3: Return Results
MATCH (activity:ACTIVITY)
RETURN activity.name AS activityName, activity.latestStart, activity.latestFinish
ORDER BY activity.depth DESC;
"""

topological order

In [ ]:
"""// for each activity match all its predecessors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) as predecessors

// for each activity find its depth for topological order
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, COALESCE(MAX(LENGTH(path)), 0) AS depth, SIZE(predecessors) AS incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC

// return results
RETURN activity.name as name, depth, incomingDependencies"""